# Sub-groups
The index space of an ND-Range kernel is divided into work-groups, sub-groups, and work-items. A work-item is the basic unit. A collection of work-items form a sub-group, and a collection of sub-groups form a work-group. The mapping of work-items and work-groups to hardware execution units (EU) is implementation-dependent. All the work-groups run concurrently but may be scheduled to run at different times depending on availability of resources. Work-group execution may or or may not be preempted depending on the capabilities of underlying hardware. Work-items in the same work-group are guaranteed to run concurrently. Work-items in the same sub-group may have additional scheduling guarantees and have access to additional functionality.

A sub-group is a collection of contiguous work-items in the global index space that execute in the same EU thread. When the device compiler compiles the kernel, multiple work-items are packed into a sub-group by vectorization so the generated SIMD instruction stream can perform tasks of multiple work-items simultaneously. Properly partitioning work-items into sub-groups can make a big performance difference.

In this section we cover performance impact and consideration when writing kernel with Sub-Groups:

- [Sub-group Sizes](#Sub-group-Sizes)
- [Sub-group Size vs. Maximum Sub-group Size](#Sub-group-Size-vs.-Maximum-Sub-group-Size)
- [Vectorization and Memory Access](#Vectorization-and-Memory-Access)
- [Data Sharing](#Data-Sharing)

## Sub-group Sizes

By default, the compiler selects a sub-group size using device-specific information and a few heuristics. The user can override the compiler’s selection using the kernel attribute `intel::reqd_sub_group_size` to specify the maximum sub-group size. Sometimes, not always, explicitly requesting a sub-group size may help performance.

The code below is sub-group example which prints sub-group information for each work-item:

In [1]:
%%writefile lab/sg_size.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  sycl::queue q;
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>()<< "\n";

  q.submit([&](auto &h) {
    sycl::stream out(65536, 256, h);
    h.parallel_for(sycl::nd_range<1>(32,32), [=](sycl::nd_item<1> it) {
         int groupId = it.get_group(0);
         int globalId = it.get_global_linear_id();
         auto sg = it.get_sub_group();
         int sgSize = sg.get_local_range()[0];
         int sgGroupId = sg.get_group_id()[0];
         int sgId = sg.get_local_id()[0];

         out << "globalId = " << sycl::setw(2) << globalId
             << " groupId = " << groupId
             << " sgGroupId = " << sgGroupId << " sgId = " << sgId
             << " sgSize = " << sycl::setw(2) << sgSize
             << sycl::endl;
    });
  });

  q.wait();
  return 0;
}


Overwriting lab/sg_size.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [2]:
! ./q.sh run_sg_size.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:02:40 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127231.v-qsvr-1           ...on_atomics.sh u49991                 0 R batch          
2127232.v-qsvr-1           run_sg_size.sh   u49991                 0 Q batch          

Waiting for Output ██████████████████████████████████████████████████████████████████████

TimeOut 60 seconds: Job is still queued for execution, check for output file later (run_sg_size.sh.o2127232)

 Done⬇

########################################################################
#      Date:           Thu 12 Jan 2023 10:18:15 AM PST
#    Job ID:           2127232.v-qsvr-1.aidevcloud
#      User:     

Each sub-group in this example has 16 work-items, or the sub-group size is 16. This means each thread simultaneously executes 16 work-items and 32 work-items are executed by two EU threads.

The valid sub-group sizes are device dependent. You can query the device to get this information:
```cpp
  std::cout << "Sub-group Sizes: ";
  for (const auto &s :
       q.get_device().get_info<sycl::info::device::sub_group_sizes>()) {
    std::cout << s << " ";
  }
  std::cout << std::endl;
```
The valid sub-group sizes supported may be:
```
Subgroup Sizes: 8 16 32
```

You can modify and check the output of the above code by overriding the compiler's selection of sub-group size, and set sub-group size to `32` using the kernel attribute `intel::reqd_sub_group_size`:
```cpp
    h.parallel_for(sycl::nd_range<1>(32,32), [=](sycl::nd_item<1> it) [[intel::reqd_sub_group_size(32)]] {
        // Kernel Code
    });
```




## Sub-group Size vs. Maximum Sub-group Size
So far in our examples, the work-group size is divisible by the sub-group size and both the work-group size and the sub-group size (either required by the user or automatically picked by the compiler are powers of two). The sub-group size and maximum sub-group size are the same if the work-group size is divisible by the maximum sub-group size and both sizes are powers of two. But what happens if the work-group size is not divisible by the sub-group size? 

Consider the following example, the sub-group size is seven, though the maximum sub-group size is still eight! The maximum sub-group size is actually the SIMD width so it does not change, but there are less than eight work-items in the sub-group, so the sub-group size is seven. So be careful when your work-group size is not divisible by the maximum sub-group size. The last sub-group with fewer work-items may need to be specially handled.

In [3]:
%%writefile lab/sg_max_size.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

constexpr int N = 15;

int main() {
  sycl::queue q;
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>()<< "\n";

  int *data = sycl::malloc_shared<int>(N + N + 2, q);

  for (int i = 0; i < N + N + 2; i++) {
    data[i] = i;
  }

  // Snippet begin
  auto e = q.submit([&](auto &h) {
    sycl::stream out(65536, 128, h);
    h.parallel_for(
        sycl::nd_range<1>(15, 15), [=](sycl::nd_item<1> it) [[intel::reqd_sub_group_size(16)]] {
          int i = it.get_global_linear_id();
          auto sg = it.get_sub_group();
          int sgSize = sg.get_local_range()[0];
          int sgMaxSize = sg.get_max_local_range()[0];
          int sId = sg.get_local_id()[0];
          int j = data[i];
          int k = data[i + sgSize];
          out << "globalId = " << i << " sgMaxSize = " << sgMaxSize
              << " sgSize = " << sgSize << " sId = " << sId << " j = " << j
              << " k = " << k << sycl::endl;
        });
  });
  q.wait();
  return 0;
}


Overwriting lab/sg_max_size.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [4]:
! ./q.sh run_sg_max_size.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:02:41 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127233.v-qsvr-1           ..._data_type.sh u49991                 0 R batch          
2127237.v-qsvr-1           ...g_max_size.sh u49991                 0 Q batch          

Waiting for Output ██████████████████████████████ Done⬇

########################################################################
#      Date:           Thu 12 Jan 2023 10:19:27 AM PST
#    Job ID:           2127237.v-qsvr-1.aidevcloud
#      User:           u49991
# Resources:           cput=75:00:00,neednodes=1:gen9:ppn=2,nodes=1:gen9:ppn=2,walltime=06:00:00
##########################################

## Vectorization and Memory Access
The Intel® graphics device has multiple EUs. Each EU is a multithreaded SIMD processor. The compiler generates SIMD instructions to pack multiple work-items in a sub-group to execute simultaneously in an EU thread. The SIMD width (thus the sub-group size), selected by the compiler is based on device characteristics and heuristics, or requested explicitly by the kernel, and can be 8, 16, or 32.

Given a SIMD width, maximizing SIMD lane utilization gives optimal instruction performance. If one or more lanes (or kernel instances or work items) diverge, the thread executes both branch paths before the paths merge later, increasing the dynamic instruction count. SIMD divergence negatively impacts performance. The compiler works to minimize divergence, but it helps to avoid divergence in the source code, if possible.
How memory is accessed in work-items affects how memory is accessed in the sub-group or how the SIMD lanes are utilized. Accessing contiguous memory in a work-item is often not optimal. 

For example: This kernel copies an array of 1024 x 1024 integers to another integer array of the same size. Each work-item copies 16 contiguous integers. However, the reads from data2 are gathered and stores to data are scattered.

In [5]:
%%writefile lab/sg_mem_access_0.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  sycl::queue q{sycl::property::queue::enable_profiling{}};

  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  constexpr int N = 1024 * 1024;
  int *data = sycl::malloc_shared<int>(N, q);
  int *data2 = sycl::malloc_shared<int>(N, q);
  memset(data2, 0xFF, sizeof(int) * N);

  auto e = q.submit([&](auto &h) {
    h.parallel_for(sycl::nd_range(sycl::range{N / 16}, sycl::range{32}),
                   [=](sycl::nd_item<1> it) {
                     int i = it.get_global_linear_id();
                     i = i * 16;
                     for (int j = i; j < (i + 16); j++) {
                       data[j] = data2[j];
                     }
                   });
  });

  q.wait();
  std::cout << "Kernel time = " << (e.template get_profiling_info< sycl::info::event_profiling::command_end>() - e.template get_profiling_info< sycl::info::event_profiling::command_start>())<< " ns\n";
  return 0;
}


Overwriting lab/sg_mem_access_0.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [6]:
! ./q.sh run_sg_mem_access_0.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:02:45 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127233.v-qsvr-1           ..._data_type.sh u49991          00:00:31 R batch          
2127240.v-qsvr-1           ...duction_sg.sh u49991                 0 R batch          
2127241.v-qsvr-1           ...m_access_0.sh u49991                 0 Q batch          

Waiting for Output ██████████████████████████████ Done⬇

########################################################################
#      Date:           Thu 12 Jan 2023 10:19:58 AM PST
#    Job ID:           2127241.v-qsvr-1.aidevcloud
#      User:           u49991
# Resources:           cput=75:00:00,neednodes=1:gen

It will be more efficient to change the code to read and store contiguous integers in each sub-group instead of each work-item. The example below does this:

In [7]:
%%writefile lab/sg_mem_access_1.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  sycl::queue q{sycl::property::queue::enable_profiling{}};

  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  constexpr int N = 1024 * 1024;
  int *data = sycl::malloc_shared<int>(N, q);
  int *data2 = sycl::malloc_shared<int>(N, q);
  memset(data2, 0xFF, sizeof(int) * N);

  auto e = q.submit([&](auto &h) {
    h.parallel_for(sycl::nd_range(sycl::range{N / 16}, sycl::range{32}),
                   [=](sycl::nd_item<1> it) {
                     int i = it.get_global_linear_id();
                     sycl::ext::oneapi::sub_group sg = it.get_sub_group();
                     int sgSize = sg.get_local_range()[0];
                     i = (i / sgSize) * sgSize * 16 + (i % sgSize);
                     for (int j = 0; j < sgSize * 16; j += sgSize) {
                       data[i + j] = data2[i + j];
                     }
                   });
  });

  q.wait();
  std::cout << "Kernel time = " << (e.template get_profiling_info<sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>()) << " ns\n";
  return 0;
}


Overwriting lab/sg_mem_access_1.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [8]:
! ./q.sh run_sg_mem_access_1.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:02:54 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127233.v-qsvr-1           ..._data_type.sh u49991          00:01:17 R batch          
2127242.v-qsvr-1           ...ion_global.sh u49991                 0 R batch          
2127243.v-qsvr-1           ...ction_sycl.sh u49991                 0 R batch          
2127244.v-qsvr-1           ...m_access_1.sh u49991                 0 Q batch          

Waiting for Output █████████████████████████████████████████ Done⬇

########################################################################
#      Date:           Thu 12 Jan 2023 10:20:29 AM PST
#    Job ID:           2127244.v-qs

Intel® graphics have instructions optimized for memory block loads/stores. So if work-items in a sub-group access a contiguous block of memory, you can use the sub-group block access functions to take advantage of these block load/store instructions.

In [9]:
%%writefile lab/sg_mem_access_2.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  sycl::queue q{sycl::property::queue::enable_profiling{}};
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  constexpr int N = 1024 * 1024;
  int *data = sycl::malloc_shared<int>(N, q);
  int *data2 = sycl::malloc_shared<int>(N, q);
  memset(data2, 0xFF, sizeof(int) * N);

  auto e = q.submit([&](auto &h) {
    h.parallel_for(
        sycl::nd_range(sycl::range{N / 16}, sycl::range{32}), [=
    ](sycl::nd_item<1> it) [[intel::reqd_sub_group_size(16)]] {
          sycl::ext::oneapi::sub_group sg = it.get_sub_group();
          sycl::vec<int, 8> x;

          using global_ptr =
              sycl::multi_ptr<int, sycl::access::address_space::global_space>;
          int base = (it.get_group(0) * 32 +
                      sg.get_group_id()[0] * sg.get_local_range()[0]) *
                     16;
          x = sg.load<8>(global_ptr(&(data2[base + 0])));
          sg.store<8>(global_ptr(&(data[base + 0])), x);
          x = sg.load<8>(global_ptr(&(data2[base + 128])));
          sg.store<8>(global_ptr(&(data[base + 128])), x);
        });
  });

  q.wait();
  std::cout << "Kernel time = " << (e.template get_profiling_info<sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>()) << " ns\n";
  return 0;
}


Overwriting lab/sg_mem_access_2.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [10]:
! ./q.sh run_sg_mem_access_2.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:03:03 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127233.v-qsvr-1           ..._data_type.sh u49991          00:02:02 R batch          
2127246.v-qsvr-1           ...ycl_blocks.sh u49991                 0 R batch          
2127247.v-qsvr-1           ...lution_slm.sh u49991                 0 R batch          
2127248.v-qsvr-1           ...m_access_2.sh u49991                 0 Q batch          

Waiting for Output ██████████████████████████████ Done⬇

########################################################################
#      Date:           Thu 12 Jan 2023 10:21:11 AM PST
#    Job ID:           2127248.v-qsvr-1.aidevc

## Data Sharing
Because the work-items in a sub-group execute in the same thread, it is more efficient to share data between work-items, even if the data is private to each work-item. Sharing data in a sub-group is more efficient than sharing data in a work-group using shared local memory, or SLM. One way to share data among work-items in a sub-group is to use shuffle functions.

This kernel transposes a 16 x 16 matrix. It looks more complicated than the previous examples, but the idea is simple: a sub-group loads a 16 x 16 sub-matrix, then the sub-matrix is transposed using the sub-group shuffle function `sycl::select_from_group(...)`. There is only one sub-matrix and the sub-matrix is the matrix so only one sub-group is needed. A bigger matrix, say 4096 x 4096, can be transposed using the same technique: each sub-group loads a sub-matrix, then the sub-matrices are transposed using the sub-group shuffle functions. This is left to the reader as an exercise.

SYCL has multiple variants of sub-group shuffle functions available. Each variant is optimized for its specific purpose on specific devices. It is always a good idea to use these optimized functions (if they fit your needs) instead of creating your own.

In [11]:
%%writefile lab/sg_shuffle.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>
#include <iomanip>

constexpr size_t N = 16;

int main() {
  sycl::queue q{sycl::property::queue::enable_profiling{}};
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  std::vector<unsigned int> matrix(N * N);
  for (int i = 0; i < N * N; ++i) {
    matrix[i] = i;
  }

  std::cout << "Matrix: " << std::endl;
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      std::cout << std::setw(3) << matrix[i * N + j] << " ";
    }
    std::cout << std::endl;
  }

  {
    constexpr size_t blockSize = 16;
    sycl::buffer<unsigned int, 2> m(matrix.data(), sycl::range<2>(N, N));

    auto e = q.submit([&](auto &h) {
      sycl::accessor marr(m, h);
      sycl::local_accessor<unsigned int, 2> barr1(sycl::range<2>(blockSize, blockSize), h);
      sycl::local_accessor<unsigned int, 2> barr2(sycl::range<2>(blockSize, blockSize), h);

      h.parallel_for(
          sycl::nd_range<2>(sycl::range<2>(N / blockSize, N),
                            sycl::range<2>(1, blockSize)),
          [=](sycl::nd_item<2> it) [[intel::reqd_sub_group_size(16)]] {
            int gi = it.get_group(0);
            int gj = it.get_group(1);

            sycl::sub_group sg = it.get_sub_group();
            int sgId = sg.get_local_id()[0];

            unsigned int bcol[blockSize];
            int ai = blockSize * gi;
            int aj = blockSize * gj;

            for (int k = 0; k < blockSize; k++) {
              bcol[k] = sg.load(marr.get_pointer() + (ai + k) * N + aj);
            }

            unsigned int tcol[blockSize];
            for (int n = 0; n < blockSize; n++) {
              if (sgId == n) {
                for (int k = 0; k < blockSize; k++) {
                  tcol[k] = sycl::select_from_group(sg, bcol[n], k);
                }
              }
            }

            for (int k = 0; k < blockSize; k++) {
              sg.store(marr.get_pointer() + (ai + k) * N + aj, tcol[k]);
            }
          });
    });
    q.wait();

    size_t kernel_time = (e.template get_profiling_info< sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>());
    std::cout << "\nKernel Execution Time: " << kernel_time * 1e-6 << " msec\n";
  }

  std::cout << std::endl << "Transposed Matrix: " << std::endl;
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      std::cout << std::setw(3) << matrix[i * N + j] << " ";
    }
    std::cout << std::endl;
  }

  return 0;
}


Overwriting lab/sg_shuffle.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [12]:
! ./q.sh run_sg_shuffle.sh gen9

Job has been submitted to Intel(R) DevCloud and will execute soon.

Executing on "gen9" node

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2127059.v-qsvr-1           ...ub-singleuser u49991          00:03:03 R jupyterhub     
2127071.v-qsvr-1           STDIN            u49991          00:02:23 R batch          
2127233.v-qsvr-1           ..._data_type.sh u49991          00:02:02 R batch          
2127250.v-qsvr-1           ...on_atomics.sh u49991                 0 R batch          
2127251.v-qsvr-1           ..._data_type.sh u49991                 0 R batch          
2127252.v-qsvr-1           ...am_256_int.sh u49991                 0 R batch          
2127253.v-qsvr-1           ...sg_shuffle.sh u49991                 0 Q batch          

Waiting for Output ██████████████████████████████ Done⬇

########################################################################
#      Date:  

## Resources

- [Intel GPU Optimization Guide](https://www.intel.com/content/www/us/en/develop/documentation/oneapi-gpu-optimization-guide/top.html) - Up to date resources for Intel GPU Optimization
- [SYCL Specification](https://registry.khronos.org/SYCL/specs/sycl-2020/pdf/sycl-2020.pdf) - Latest Specification document for reference
- [SYCL Essentials Training](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL/Jupyter/oneapi-essentials-training) - Learn basics of C++ SYCL Programming